In [1]:
from load_data import load_expert_data
from utils_analysis import sort_by_key

run = 4
n_q = '*'
batch = '*'
group = 'reason_agreement_expert_inspection1'
expert_data = load_expert_data(run, group, n_q, batch)
print(len(expert_data))

run4-group_reason_agreement_expert_inspection1/qu30-s_qu30-batch1.csv
no summary data
33


In [11]:
data_by_triple = sort_by_key(expert_data, ['description'])

for triple, data in data_by_triple.items():
    print('question: ', triple)
    for d in data:
        print(d['workerid'], d['answer'], d['crowd'])
        print('---')
        if 'reason' in d:
            print('reason:', d['reason'])
            print('---')
        if 'comment' in d:
            print('comment', d['comment'])
    print()

question:  ``Arrow'' is one of the first things which come to mind when I hear ``fly' because (a/an) arrow is a typical example of things which fly/are used for flying'. 
pia_run2 false true
---

question:  ``Fly'' is one for the first things which come to mind when I hear ``arrow' because flying is one of the typical movements of (a/an) arrow'.
pia_run2 true true
---

question:  I know that all or most arrow(s) fly regularly or are used for flying regularly. 
pia_run2 true false
---
reason: I think the question is ambiguous. `Regularly' could be read in the sense of `frequently' instead of `main purpose'. Some people might think that most of the time, arrows are stored/carried around with or stuck in a target. The time they spend flying is rather little.
---

question:  You can find (a/an) arrow which can fly/be used for flying. This distinguishes a certain type of arrow from others. 
pia_run1 false true
---
pia_run2 false true
---

question:  I could say (a/an) arrow flies/is used fo